In [ ]:
!pip install bertopic  #importing bertopic
!pip install sentence-transformers #importing seentence transfomers
import pandas as pd #importing pandas
import numpy as np #importing numpy
import re #importing re
import string  #importing string

from nltk.corpus import wordnet #importing wordnet
from nltk.tokenize import word_tokenize #importing word tokenize
from nltk.stem import WordNetLemmatizer #importing wordnetlemmatizer
import nltk #import nltk
nltk.download('averaged_perceptron_tagger') 
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
from sentence_transformers import SentenceTransformer #import sentencetransfomer

In [ ]:
#reading the dataset 
df= pd.read_csv('/content/all-data (1).csv',header=None,
                 names=['sentiment','News'],encoding='latin-1')
df.shape #shape of the  dataset

(4846, 2)

In [ ]:
#convert to lowercase, strip and remove punctuations
def preprocess(text):
    text = text.lower() 
    text=text.strip()  
    text=re.compile('<.*?>').sub('', text)
    text=re.compile('[a-zA-Z0-9-_.]+@[a-zA-Z0-9-_.]+').sub('', text) 
    text=re.compile('((25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)(\.|$)){4}').sub('', text)
    text = re.compile('[%s]' % re.escape(string.punctuation)).sub(' ', text)  
    text = re.sub('\s+', ' ', text)  
    text = re.sub(r'\[[0-9]*\]',' ',text) 
    text=re.sub(r'[^\w\s]', '', str(text).lower().strip())
    text = re.sub(r'\d',' ',text) 
    text = re.sub(r'\s+',' ',text)
    
    return text


  
# STOPWORD REMOVAL
def stopword(string):
    a= [i for i in string.split() if i not in stopwords.words('english')]
    return ' '.join(a)



#LEMMATIZATION
# Initialize the lemmatizer


from nltk.corpus import stopwords 
wl = WordNetLemmatizer()
 
# This is a helper function to map NTLK position tags
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN
# Tokenize the sentence
def lemmatizer(string):
    word_pos_tags = nltk.pos_tag(word_tokenize(string)) # Get position tags
    a=[wl.lemmatize(tag[0], get_wordnet_pos(tag[1])) for idx, tag in enumerate(word_pos_tags)] # Map the position tag and lemmatize the word/token
    return " ".join(a)


def finalpreprocess(string):
    return lemmatizer(stopword(preprocess(string)))

In [ ]:
df['Clean_text'] = df['News'].apply(lambda x: finalpreprocess(x)) #applying the preprocess function on the data
df['Clean_text']

0       accord gran company plan move production russi...
1       technopolis plan develop stage area less squar...
2       international electronic industry company elco...
3       new production plant company would increase ca...
4       accord company update strategy year basware ta...
                              ...                        
4841    london marketwatch share price end lower londo...
4842    rinkuskiai beer sale fell per cent million lit...
4843    operating profit fell eur mn eur mn include ve...
4844    net sale paper segment decrease eur mn second ...
4845    sale finland decreased january sale outside fi...
Name: Clean_text, Length: 4846, dtype: object

In [ ]:
df=df.drop_duplicates(subset = ['Clean_text'],keep = 'last').reset_index(drop = True)
from collections import OrderedDict

df['Desired'] = (df['Clean_text'].str.split().apply(lambda x: OrderedDict.fromkeys(x).keys()).str.join(' '))

In [ ]:
df['Desired']=df['Desired'].str.replace(r'\b\w\b', '').str.replace(r'\s+', ' ')

In [ ]:
!pip install numpy

In [ ]:
from bertopic import BERTopic #(if it shows error run pip install numpy again)
import gensim.corpora as corpora
from gensim.models.coherencemodel import CoherenceModel
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

import operator

In [49]:
def bertopic_model(X):
  vectorizer = CountVectorizer(analyzer='word',       
                              min_df=10,                        # minimum reqd occurences of a word 
                              stop_words='english',             # remove stop words
                              lowercase=True,                   # convert all words to lowercase
                              token_pattern='[a-zA-Z0-9]{3,}',  # num chars > 3
                            )
  sentence_model = SentenceTransformer("paraphrase-MiniLM-L3-v2") #loading the model 
  embeddings = sentence_model.encode(X, show_progress_bar=True) #creating emdedding 
  # Create topic model
  topic_model = BERTopic(verbose=True,top_n_words=20,nr_topics='auto',calculate_probabilities=True) #saving the bertopic with variable
  topics, probs = topic_model.fit_transform(X,embeddings) #fit_transform the text and probabilities

  freq = topic_model.get_topic_info()  #frequncey of the topics 

  barchart=topic_model.visualize_barchart(n_words=10,top_n_topics=15) #top 15 words in top 10 topics 

  topic_map=topic_model.visualize_topics()  #visualization of the topics 



  # Preprocess documents
  cleaned_docs = topic_model._preprocess_text(X)

  # Extract vectorizer and tokenizer from BERTopic
  vectorizer = topic_model.vectorizer_model
  tokenizer = vectorizer.build_tokenizer()

  # Extract features for Topic Coherence evaluation
  words = vectorizer.get_feature_names()
  tokens = [tokenizer(doc) for doc in cleaned_docs]
  dictionary = corpora.Dictionary(tokens)
  corpus = [dictionary.doc2bow(token) for token in tokens]
  topic_words = [[words for words, _ in topic_model.get_topic(topic)] 
                for topic in range(len(set(topics))-1)]

  # Evaluate
  coherence_model = CoherenceModel(topics=topic_words, 
                                  texts=tokens, 
                                  corpus=corpus,
                                  dictionary=dictionary, 
                                  coherence='c_v')
  coherence = coherence_model.get_coherence()

  

  return topics, probs,freq,barchart,topic_map,coherence

In [51]:
topics, probs,freq,barchart,topic_map,coherence=bertopic_model(df['Desired'])

Batches:   0%|          | 0/149 [00:00<?, ?it/s]

2022-01-31 14:02:10,464 - BERTopic - Reduced dimensionality with UMAP
2022-01-31 14:02:12,728 - BERTopic - Clustered UMAP embeddings with HDBSCAN
2022-01-31 14:02:13,458 - BERTopic - Reduced number of topics from 94 to 68


In [52]:
barchart

In [53]:
topic_map

In [54]:
coherence

0.4001215954267481